# Submit Notebook

- road
- building

の両方の結果を合わせて出力する

## Config

In [43]:
ROAD_CSV = '../../train/output/foundation/A_V7_FINE_clean-v4__timmu_IMG1312_fl-tr_b4/A_fold-4/solution_road.csv'
BUILD_CSV = '../../train/output/foundation/A_V7_FINE_clean-v4__timmu_IMG1312_fl-tr_b4/A_fold-4/test_fnd/solution_building.csv'

SUBMIT_SCV = '../../submit/my_submit/solution/solution.csv'

## Run

In [44]:
import pandas as pd

# submit_colnmns = ['ImageId', 'Object', 'Flooded', 'Wkt_Pix', 'Wkt_Geo']
submit_colnmns = ['ImageId', 'Object', 'WKT_Pix', 'Flooded']

In [46]:
dfs = []

df = pd.read_csv(ROAD_CSV)
df = df[submit_colnmns + ['length_m', 'travel_time_s']]
dfs.append(df)

df = pd.read_csv(BUILD_CSV).rename(columns={'Wkt_Pix': 'WKT_Pix', 'Wkt_Geo': 'WKT_Geo'})
df = df[submit_colnmns]
df['length_m'] = 'null'
df['travel_time_s'] = 'null'
dfs.append(df)
    
df = pd.concat(dfs)

# 重複削除
check_col = submit_colnmns + ['length_m', 'travel_time_s']

df_dup = df.duplicated(subset=check_col)
num_dup = len(df_dup == False)
print(f"重複: {num_dup}")
df_sub = df[~df_dup]

df_sub.head(3)

重複: 9365


,ImageId,Object,WKT_Pix,Flooded,length_m,travel_time_s
0,104001006504F400_0_0_17,Road,"LINESTRING (1006 1, 978 21)",False,11.439228,3.412
1,104001006504F400_0_0_17,Road,"LINESTRING (823 119, 689 206)",False,52.798634,15.748
2,104001006504F400_0_0_17,Road,"LINESTRING (257 514, 305 585, 441 747, 537 855...",False,215.37972,64.239


## Point Duplicated
road で重複がたまに発生してるので確認

In [47]:
def remove_duplicated(linestring):
    
    if linestring != 'POLYGON EMPTY':
        # exist object
        if 'LINESTRING' in linestring:
            points = linestring.split('(')[1].split(')')[0]
            # print(points)
            
            knowns = []
            for i, xy in enumerate(points.split(',')):
                if i == 0:
                    knowns.append(' ' + xy)
                else:
                    if not xy in knowns:
                        knowns.append(xy) 
                    else:
                        print('◆'*20, f'delete duplicated {xy} <<< {linestring}')
            
            # top space
            knowns[0] = knowns[0][1:]
            linestring = 'LINESTRING (' + ','.join(knowns) + ')'
    return linestring
               
df_sub['WKT_Pix'] = df_sub['WKT_Pix'].apply(remove_duplicated)
df_sub.head(3)

◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  196 3 <<< LINESTRING (196 3, 199 5, 201 0, 196 3)
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  238 0 <<< LINESTRING (238 0, 258 0, 243 5, 238 0)
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  943 1 <<< LINESTRING (943 1, 959 3, 951 9, 943 1)
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  428 0 <<< LINESTRING (428 0, 434 2, 431 3, 428 0)
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  1268 632 <<< LINESTRING (1268 632, 1275 617, 1293 609, 1297 602, 1297 624, 1293 630, 1268 632)
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  489 5 <<< LINESTRING (489 5, 496 0, 498 3, 489 5)
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  694 3 <<< LINESTRING (694 3, 697 3, 695 1, 694 3)
◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆ delete duplicated  352 1146 <<< LINESTRING (352 1146, 377 1153, 401 1155, 424 1172, 352 1146)


,ImageId,Object,WKT_Pix,Flooded,length_m,travel_time_s
0,104001006504F400_0_0_17,Road,"LINESTRING (1006 1, 978 21)",False,11.439228,3.412
1,104001006504F400_0_0_17,Road,"LINESTRING (823 119, 689 206)",False,52.798634,15.748
2,104001006504F400_0_0_17,Road,"LINESTRING (257 514, 305 585, 441 747, 537 855...",False,215.37972,64.239


In [48]:
df_sub.to_csv(SUBMIT_SCV, index=False, header=True)

In [49]:
df[df_dup]

,ImageId,Object,WKT_Pix,Flooded,length_m,travel_time_s


In [50]:
df_dup.value_counts()

False    9365
dtype: int64